In [1]:
# Stage 1: Search a template

In [2]:
def code2pirseq(code):
    from Bio import ExPASy, SeqIO
    spraw = ExPASy.get_sprot_raw(code)
    spinfo = SeqIO.read(spraw, "swiss")
    PDBs = [x[4:] for x in spinfo.dbxrefs if x.startswith('PDB:')]
    #print len(PDBs), PDBs
    
    # Name of the protein (a name without a white space is preferred)
    name     = str(spinfo.name)
    sequence = str(spinfo.seq)
    outname  = "%s.ali" % name
    with open(outname, "w") as f:
        f.write(">P1;%s\n" % name)
        f.write("sequence:%s: %d:A:%d: A:::0.00: 0.00\n" % (name, 1, len(sequence)) )
        f.write("%s*\n" % sequence)
    return name, outname, sequence

In [4]:
# Verificar que obtener un archivo de nuestra secuencia de inetres en formato PIR
code = "P11167"
name, outfile, sequence = code2pirseq(code)
print(outfile)

GTR1_RAT.ali


In [7]:
!wget https://salilab.org/modeller/downloads/pdb_95.pir.gz

--2022-04-12 16:22:23--  https://salilab.org/modeller/downloads/pdb_95.pir.gz
Resolviendo salilab.org (salilab.org)... 169.230.79.19
Conectando con salilab.org (salilab.org)[169.230.79.19]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 17047044 (16M) [application/x-gzip]
Grabando a: «pdb_95.pir.gz»

pdb_95.pir.gz       100%[===================>]  16.26M  2.62MB/s    en 8.8s    

2022-04-12 16:22:33 (1.84 MB/s) - «pdb_95.pir.gz» guardado [17047044/17047044]



In [8]:
# Para obetenr la base de datos de PDB no redundantes desde Modeller
#!wget https://salilab.org/modeller/downloads/pdb_95.pir.gz

def findTemplate(spcode):
    name, outname, sequence = code2pirseq(spcode)
    pdb_95_pir = "pdb_95.pir.gz"
    seqQuery   = outname
    
    from modeller import environ, log, alignment, sequence_db
    
    log.verbose()
    env = environ()
    
    #-- Prepare the input files
    
    #-- Read in the sequence database
    sdb = sequence_db(env)
    sdb.read(seq_database_file=pdb_95_pir, seq_database_format='PIR',
             chains_list='ALL', minmax_db_seq_len=(30, 4000), clean_sequences=True)
    
    #-- Write the sequence database in binary form
    sdb.write(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
              chains_list='ALL')
    
    #-- Now, read in the binary database
    sdb.read(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
             chains_list='ALL')
    
    #-- Read in the target sequence/alignment
    aln = alignment(env)
    aln.append(file=seqQuery, alignment_format='PIR', align_codes='ALL')
    
    #-- Convert the input sequence/alignment into
    #   profile format
    prf = aln.to_profile()
    
    #-- Scan sequence database to pick up homologous sequences
    prf.build(sdb, matrix_offset=-450, rr_file='${LIB}/blosum62.sim.mat',
              gap_penalties_1d=(-500, -50), n_prof_iterations=1,
              check_profile=False, max_aln_evalue=0.01)
    
    #-- Write out the profile in text format
    prf.write(file='%s_profile.prf' % name, profile_format='TEXT')
    
    #-- Convert the profile back to alignment format
    aln = prf.to_alignment()
    
    #-- Write out the alignment file
    aln.write(file='%s_profile.ali' % name, alignment_format='PIR')

In [9]:
findTemplate(spcode="P11167")

environ____W> The class 'environ' is deprecated; use 'Environ' instead
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v1}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       445980     435.527     0.425

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       446508     436.043     0.426
openf___224_> Open           ${MODINSTALL10v1}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       495108     483.504     0.472
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v1}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,

In [11]:
!wget https://files.rcsb.org/download/4zw9.pdb

--2022-04-12 16:25:53--  https://files.rcsb.org/download/4zw9.pdb
Resolviendo files.rcsb.org (files.rcsb.org)... 128.6.158.70
Conectando con files.rcsb.org (files.rcsb.org)[128.6.158.70]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: no especificado [application/octet-stream]
Grabando a: «4zw9.pdb»

4zw9.pdb                [     <=>            ] 654.57K   644KB/s    en 1.0s    

2022-04-12 16:25:54 (644 KB/s) - «4zw9.pdb» guardado [670275]



In [12]:
from modeller import *
#Alineamiento de la secuencia y el templado.
env = environ()
aln = alignment(env)
mdl = model(env, file='4ZW9', model_segment=('FIRST:A','LAST:A'))
aln.append_model(mdl, align_codes='4ZW9', atom_files='4ZW9.pdb')
aln.append(file='GTR1_RAT.ali', align_codes='GTR1_RAT')
aln.align2d(max_gap_length=40)
aln.write(file='GTR1_RAT-4ZW9.ali', alignment_format='PIR')
aln.write(file='GTR1_RAT-4ZW9.pap', alignment_format='PAP')

environ____W> The class 'environ' is deprecated; use 'Environ' instead
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v1}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       445980     435.527     0.425

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       446508     436.043     0.426
openf___224_> Open           ${MODINSTALL10v1}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       495108     483.504     0.472
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v1}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,

In [13]:
from modeller import *
from modeller.automodel import *
#from modeller import soap_protein_od

env = environ()
a = automodel(env, alnfile='GTR1_RAT-4ZW9.ali',knowns='4ZW9', sequence='GTR1_RAT',
              assess_methods=(assess.DOPE,assess.GA341))
a.starting_model = 1
a.ending_model = 3 #va a presentar hasta 3 modelos diferentes
a.make()

environ____W> The class 'environ' is deprecated; use 'Environ' instead
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v1}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      1789967    1748.015     1.707

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      1790495    1748.530     1.708
openf___224_> Open           ${MODINSTALL10v1}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      1839095    1795.991     1.754
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v1}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,

In [20]:
# Evaluar el modelo con DOPE, se obtienen lso perfiles de pseudo energia
from modeller import *
from modeller.scripts import complete_pdb

log.verbose()    # request verbose output
env = environ()
env.libs.topology.read(file='$(LIB)/top_heav.lib') # read topology
env.libs.parameters.read(file='$(LIB)/par.lib') # read parameters

# read model file
mdl = complete_pdb(env, 'GTR1_HUMAN.B99990003.pdb')

# Assess with DOPE:
s = selection(mdl)   # all atom selection
s.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='GTR1_HUMAN_M3.profile',
              normalize_profile=True, smoothing_window=15)

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v24}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v24}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40395332   39448.566    38.524

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40395860   39449.082    38.524
openf___224_> Open           ${MODINSTALL9v24}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40444460   39496.543    38.571
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL9v24}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40457760   39509.531    38.584
openf___224_> Open           $

-67534.6328125

In [21]:
#Evaluar el templado con DOPE, se obiene el perfil de pseudo energia
log.verbose()    # request verbose output
env = environ()
env.libs.topology.read(file='$(LIB)/top_heav.lib') # read topology
env.libs.parameters.read(file='$(LIB)/par.lib') # read parameters

# directories for input atom files
env.io.atom_files_directory = './:../atom_files'

# read model file
mdl = complete_pdb(env, '4ZW9.pdb', model_segment=('FIRST:A', 'LAST:A'))

s = selection(mdl)
s.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='4ZW9.profile',
              normalize_profile=True, smoothing_window=15)

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v24}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v24}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40395332   39448.566    38.524

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40395860   39449.082    38.524
openf___224_> Open           ${MODINSTALL9v24}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40444460   39496.543    38.571
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL9v24}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     40457760   39509.531    38.584
openf___224_> Open           $

-70655.390625

In [13]:
import pylab
import modeller

def r_enumerate(seq):
    """Enumerate a sequence in reverse order"""
    # Note that we don't use reversed() since Python 2.3 doesn't have it
    num = len(seq) - 1
    while num >= 0:
        yield num, seq[num]
        num -= 1

def get_profile(profile_file, seq):
    """Read `profile_file` into a Python array, and add gaps corresponding to
       the alignment sequence `seq`."""
    # Read all non-comment and non-blank lines from the file:
    f = open(profile_file, "r")
    vals = []
    for line in f:
        if not line.startswith('#') and len(line) > 10:
            spl = line.split()
            vals.append(float(spl[-1]))
    # Insert gaps into the profile corresponding to those in seq:
    for n, res in r_enumerate(seq.residues):
        for gap in range(res.get_leading_gaps()):
            vals.insert(n, None)
    # Add a gap at position '0', so that we effectively count from 1:
    vals.insert(0, None)
    return vals

e = modeller.environ()
a = modeller.alignment(e, file='GTR1_RAT-6THA.ali')

template = get_profile('6THA.profile', a['6THA'])
model = get_profile('GTR1_RAT_M1.profile', a['GTR1_RAT'])

# Plot the template and model profiles in the same plot for comparison:
pylab.figure(1, figsize=(10,6))
pylab.xlabel('Alignment position')
pylab.ylabel('DOPE per-residue score')
pylab.plot(model, color='red', linewidth=2, label='Model')
pylab.plot(template, color='green', linewidth=2, label='Template')
pylab.legend()
pylab.savefig('dope_profile.png', dpi=65)

environ____W> The class 'environ' is deprecated; use 'Environ' instead
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v1}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     26723299   26096.973    25.485

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     26723827   26097.488    25.486
openf___224_> Open           ${MODINSTALL10v1}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:     26772427   26144.949    25.532
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v1}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,

FileNotFoundError: [Errno 2] No such file or directory: '6THA.profile'